In [45]:
import pandas as pd
import datetime, os

#import win32com.client as win32
from tkinter import filedialog
from win32com.client import Dispatch

In [ ]:
def GetFilePath(*arg):
    autosvod_path = filedialog.askopenfilename()
    return autosvod_path

In [47]:
maindf = pd.read_excel(r"D:\MyProjects\RFKOD\Итог_ Детализации по ТБ_сделки_ПР_КОД_01.01.2021-17.01.2021_VerID1.xlsx", skiprows=2)
otvdf = pd.read_excel(r"D:\MyProjects\RFKOD\Рассылка РФиКОД (ПР).xlsm",usecols=[0,1])

Path_to_table = dict()
Otvetstvenie_email = dict()

In [48]:
def GetTBList(df, colummindex):
    """Получаем книгу возвращает список ТБ"""
    try:
        ndf = list(df.iloc[:,int(colummindex)].unique())
        tb_list = sorted(ndf)
        return tb_list
    except:
        return print(f"Не удалось получить dataframe или столбец по индексу {colummindex}")

In [49]:
def GetTableToSend(df,TB_list, *arg):
    """Режим таблицу по ТБ и складываем в папку Temp"""
    
    for i in TB_list:
        path_to_table = rf"C:\Temp\output_{i}.xlsx"
        df_to_send = df.loc[df["ТБ кр"] == i].copy()
        df_to_send.to_excel(path_to_table)
        Path_to_table.update({i : path_to_table})
        del df_to_send
        
    return Path_to_table

In [50]:
def OtvString(otvdf, tb_list):
    """Получаем адресатов"""

    for i in tb_list:
        sendTo = ''
        email_otv = otvdf.loc[otvdf['ТБ'] == i]['Кураторы Проекта со стороны ПКД   Модуля ППРБ ']
        
        for i in email_otv:
            sendTo = sendTo + str(i) + '; '
        Otvetstvenie_email.update({i:sendTo})

    return Otvetstvenie_email

In [51]:
def AutoSize(file_path):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(file_path)
    excel.Visible = False
    ws = wb.Worksheets('Sheet1')
    ws.Columns.AutoFit()
    wb.Save()
    excel.Application.Quit()
    return file_path

In [52]:
tb_list = GetTBList(df,0)
Path_to_table = GetTableToSend(maindf, tb_list)
Otvetstvenie_email = OtvString(otvdf, tb_list)

In [18]:
def SendMail(options, TB, *arg):
    outlook = Dispatch('outlook.application')
    AttachFile = path_to_tb_table[TB]
    
    mail = outlook.CreateItem(0)

    mail.To = OtvString(otvdf, TB)
    mail.CC = options["CC"]
    mail.Subject = f"Тема сообщения для банка {TB}"
    mail.Body = options["Body"]
    mail.HTMLBody = ""
    mail.Attachments.Add(AttachFile)
    mail.Display(True)
    
    #os.remove(AttachFile) # Удаление файла из ФИР